In [ ]:
!pip3 install pandas_summary isoweek sklearn_pandas -i https://pypi.tuna.tsinghua.edu.cn/simple
! pip3 install --upgrade keras

In [114]:
import matplotlib.pyplot as plt
from imp import reload
import re
import pickle
import math, keras, datetime, pandas as pd, numpy as np, keras.backend as K
from pandas_summary import DataFrameSummary
import bld_etl; reload(bld_etl)
from keras.preprocessing import sequence
from bld_etl import read_raw, loadtypes, add_datepart, fillna, type_map, get_data
from sklearn.model_selection  import train_test_split

import numpy as np
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten, Dropout, Embedding, Concatenate
from keras import initializers, Model
from keras.layers.convolutional import Convolution1D
from keras.layers.pooling import MaxPooling1D
from keras import optimizers

import math, keras, datetime, pandas as pd, numpy as np, keras.backend as K
from pandas_summary import DataFrameSummary
import bld_etl; reload(bld_etl)
from bld_etl import read_raw, loadtypes, add_datepart, fillna, type_map, get_data, load_vectors, par2idx, load_var, dump_var

%matplotlib inline
np.set_printoptions(threshold=50, edgeitems=20)

In [6]:
vecs, words, wordidx = load_vectors('/root/.keras/models/fsttxtzh300')
idx2word = {v: k for k, v in wordidx.items()}

In [9]:
merged_df = pd.read_json('/data/ai_opinfo_2017-01_to_2017-10.json', lines=True)
# filtered the data without RBC
merged_df = merged_df[~merged_df.TRANS_RBC.isnull()]
y_df, x_df = read_raw(merged_df)
contin_vars, cat_vars, dt_vars, text_vars, const_vars = loadtypes(x_df)
# drop the cols with contant value, cause this contribute nothing to predicted results
x_df.drop(const_vars, axis=1, inplace=True, errors="ignore")
add_datepart(x_df, dt_vars)
fillna(x_df, contin_vars, cat_vars)
contin_vars.remove('TRANS_RBC')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [10]:
cat_map_fit, contin_map_fit, contin_cols = type_map(joined, cat_vars, contin_vars, use_cache=False, cache_dir='/data/result')

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [35]:
x_train, y_train, x_valid, y_valid, cat_map_train, cat_map_valid, contin_map_train, contin_map_valid = get_data(joined, cat_map_fit, contin_map_fit)
# cat_map_train: mapped categorical data

/notebooks/bld_etl.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  joined_train.drop('TRANS_RBC', axis=1, inplace=True)
/notebooks/bld_etl.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  joined_valid.drop('TRANS_RBC', axis=1, inplace=True)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [12]:
def rmspe(y_pred, targ = y_valid):
    pct_var = (targ - y_pred)/targ
    return math.sqrt(np.square(pct_var).mean())
# def log_max_inv(preds, mx = max_log_y):
#     return np.exp(preds * mx)
def normalize_inv(preds):
    return preds * ystd + ymean

In [34]:
# split matrix into num (arr.shape[1]) lists
def split_cols(arr): return np.hsplit(arr,arr.shape[1])

In [14]:
map_train = split_cols(cat_map_train) + [contin_map_train]
map_valid = split_cols(cat_map_valid) + [contin_map_valid]

In [155]:
import re
def cat_preproc(dat, cat_map_fit):
    "cat_preproc"
    return cat_map_fit.transform(dat).astype(np.int64)


def contin_preproc(dat, contin_map_fit):
    "contin_preproc"
    return contin_map_fit.transform(dat).astype(np.float32)

def get_data(joined, cat_map_fit, contin_map_fit, text_vars, wordidx):
    y = joined.TRANS_RBC
    joined.drop('TRANS_RBC', axis=1, inplace=True, errors='ignore')
    cat_map = cat_preproc(joined, cat_map_fit)
    contin_map = contin_preproc(joined, contin_map_fit)
    
    op_name_map = [par2idx(line, words, wordidx) for line in joined.OP_NAME]
    diag_name_map = [par2idx(line, words, wordidx) for line in joined.INHOS_DIAG_NAME]
    
    seq_len = 80
    op_name_map = sequence.pad_sequences(op_name_map, maxlen=seq_len, value=0)
    diag_name_map = sequence.pad_sequences(diag_name_map, maxlen=seq_len, value=0)
    return cat_map, contin_map, op_name_map, diag_name_map, y

In [109]:
joined = pd.read_pickle('/data/result/cleaned_data.pkl')
cat_map, contin_map, op_name_map, diag_name_map, y = get_data(joined, cat_map_fit, contin_map_fit, text_vars, wordidx)


/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [116]:
dump_var('/data/result/cat_map.pkl', cat_map)
dump_var('/data/result/contin_map.pkl', contin_map)
dump_var('/data/result/op_name_map.pkl', op_name_map)
dump_var('/data/result/diag_name_map.pkl', diag_name_map)

In [117]:
cat_map = load_var('/data/result/cat_map.pkl')
contin_map = load_var('/data/result/contin_map.pkl')
op_name_map = load_var('/data/result/op_name_map.pkl')
diag_name_map = load_var('/data/result/diag_name_map.pkl')

In [141]:
dataset = split_cols(cat_map) + [contin_map] + [op_name_map] + [diag_name_map]

In [154]:
trn = dataset
test = dataset


In [150]:
tt = split_cols(cat_map)


In [153]:
len(dataset)

11

In [148]:
y.shape

(5207,)